In [1]:
using Pkg, Revise
Pkg.activate("../GenLinAlgProblems")

using GenLinAlgProblems, LinearAlgebra, ToeplitzMatrices, DSP, SymPy, Latexify, LaTeXStrings, Random, StatsBase, IJulia

x = symbols("x", real=true)
z = [symbols("z_$i", real=true) for i in 1:7]
Random.seed!( 12345 );

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


<div style="float:center;width:100%;text-align: center;">
    <strong style="height:60px;color:darkred;font-size:40px;">Circulant Matrices</strong><br>
</div>

# 1. Non-cyclic (Linear) and Cyclic Convolution

## 1.1 Linear (non-cyclic) Convolution

### 1.1.1 Definition

**Definition**: Given two vectors $f$ and $g$ of lengths $M$ and $N$ respectively,<br>
$\qquad$ their **linear convolution** is a vector of length $M+N-1$.<br><br>
$\qquad\qquad (f * g)_n = \sum_{m=\max(0,n-N+1)}^{\min(M-1,n)}\; f_m\; g_{n-m}$<br><br>
$\qquad$ Note that the result is longer than both input vectors<br>

**Polynomial interpretation:** Linear convolution corresponds to **polynomial multiplication** in the ring of polynomials $F[X]$.

### 1.1.2 Examples

* **Example:** $f = [1, 2, 3]$ and $g = [4, 5],\;\;$ then $f*g = [4, 13, 22, 15]$<br><br>
<div style="float:left;padding-left:1cm;width:40%;">
<table style="border-collapse: collapse width: 30%;float:left;">
  <tr style="border: 1px solid black;">
    <td>f</td>
    <td><span style="display: inline-block; width: 3cm;">&nbsp;</span></td>
    <td></td>
    <td style="text-align: center;">1</td>
    <td style="text-align: center;">2</td>
    <td style="text-align: center;">3</td>
  </tr>
  <tr style="border: 1px solid black;border-bottom: 2px solid black;">
    <td>reversed g</td>
    <td></td>
    <td></td>
    <td></td>
    <td style="text-align: center;">4</td>
    <td style="text-align: center;">5</td>
  </tr>
  <tr style="border: 1px solid black;">
    <td>product f×5 </td>
    <td></td>
    <td></td>
    <td style="text-align: center;">5</td>
    <td style="text-align: center;">10</td>
    <td style="text-align: center;">15</td>
  </tr>
  <tr style="border: 1px solid black;border-bottom: 2px solid black;">
    <td>product f×4</td>
    <td></td>
    <td style="text-align: center;">4</td>
    <td style="text-align: center;">8</td>
    <td style="text-align: center;">12</td>
    <td></td>
  </tr>
  <tr style="border: 1px solid black;">
    <td>sum of products</td>
    <td></td>
    <td style="text-align: center;">4</td>
    <td style="text-align: center;">13</td>
    <td style="text-align: center;">22</td>
    <td style="text-align: center;">15</td>
  </tr>
</table>
</div><div style="float:left;padding-left:1cm;width:40%;">
The linear convolution can be represented in the same way as<br> multiplication of numbers (without carry)<br><br>
<strong>Remark:</strong> A check of the result is $\sum{f}\ \sum{g} = \sum{f*g},\;$<br>
$\qquad\qquad ( 1+2+3 ) (4+5) = ( 4+13+22 +15 )$
</div>

* **Example:** Given $\;\;f = 1+2x+3 x^2\;\;$ and $\;\;g = 4 + 5x,\;\;$ then $\;\;f*g = 4 + 13 x+22 x^2 + 15 x^3$

In [2]:
@show conv( [1 2 3], [4 5]) == [4 13 22 15];
@show (1+2x+3x^2)*(4+5x) == (4+13x+22x^2+15x^3);

conv([1 2 3], [4 5]) == [4 13 22 15] = true
(1 + 2x + 3 * x ^ 2) * (4 + 5x) == 4 + 13x + 22 * x ^ 2 + 15 * x ^ 3 = true


### 1.1.3 Matrix representation

Linear convolution can be represented as a matrix-vector multiplication<br>
$\qquad$ $z = T x,\;\;$ where $T$ is a Toeplitz matrix formed from the vector $g$.

**Definition:** A **Toeplitz matrix** or diagonal-constant matrix, is a matrix in which each descending diagonal from left to right is constant.<br>
$\qquad$ (Note that a Toeplitz Matrix need not be square.)

$\qquad T = \begin{pmatrix} \color{red}{a_0} & a_{-1} & a_{-2} & \dots & a_{-n+1} \\
                            a_{1} & \color{red}{a_{0}} & a_{-1} & \dots & a_{-n+2} \\
                            a_{2} & a_{1} &  \color{red}{\ddots} & \ddots &  \ddots  \\
                            \ddots & \ddots &  \ddots & \color{red}{\ddots} &  \ddots \\
                            a_{n-1} & a_{n-2} & \dots & \dots & \color{red}{a_0} \end{pmatrix}\qquad$ (here only the main diagonal is shown in red)

To encode a vector $g$ of length $M$ in a Toeplitz matrix for computing the linear convolution with another vector $f$ of length $N$:<br>
 (N+M-1) × N.
Construct a Toeplitz matrix $T$ of size $(N+M-1) \times N$:
* The first column of $T$ contains the vector $g$ padded with $N-1$ zeros at the end.
* Each subsequent column is a downward shift of the previous column.

**Example:** Let $g = [4, 5, 6]$, and let $f$ have length 3, e.g., $f=[1,2,3]$ Then<br><br>
$\qquad T f = \begin{pmatrix}
4 & 0 & 0 \\
5 & 4 & 0 \\
6 & 5 & 4 \\
0 & 6 & 5 \\
0 & 0 & 6 \\
\end{pmatrix} \begin{pmatrix} 1 \\ 2 \\ 3 \end{pmatrix} = \left(
\begin{array}{r}
4 \\
13 \\
28 \\
27 \\
18 \\
\end{array}
\right)$

In [3]:
function convolution_matrix(g::Vector{T}, n::Int) where T
    g_padded = [g; zeros(T, n-1)]
    Toeplitz(g_padded, [g[1]; zeros(T, n-1)])
end
f = [1; 2; 3]
g = [4,5,6]
T = convolution_matrix(g, length(f))
println("f*g = ", conv(f,g))
@show T*f == conv(f,g);

f*g = [4, 13, 28, 27, 18]
T * f == conv(f, g) = true


## 1.2 Cyclic or Circular Convolution

### 1.1.2 Definition

**Definition:** Given two vectors $f$ and $g$ both of length $N$,<br>
$\qquad$ their cyclic or **circular convolution** is a vector of length $N$<br><br>
$\qquad\qquad (f \circledast g)_n = \sum_{m=0}^{N-1}\; f_m \; h_{n-m} \bmod N,\qquad$ where $\bmod$ is the modulo operation applied to the indices

The circular convolution has the following key properties:
<div style="float:left;padding-left:1.5cm;">

| Property | Formula |
| :-------- | :------- |
| **Periodicity:** | $\qquad (f \circledast g)_n = (f \circledast g)_{n+N}$ |
| **Commutativity:** | $\qquad f \circledast g = g \circledast f$ |
| **Associativity:** | $\qquad (g \circledast g) \circledast h = f \circledast (g \circledast h)$ |
| **Distributivity over addition:** | $\qquad f \circledast (g + h) = (f \circledast g) + (f \circledast h)$ |
</div>

#### Examples

* **Example: Vectors**

In [4]:
function circular_conv(x::Vector{T}, h::Vector{T}) where T
    # naive implementation
    N = length(x)
    y = zeros(T, N)
    for n in 0:N-1
        for m in 0:N-1
            y[n+1] += x[m+1] * h[mod(n-m, N) + 1]
        end
    end
    return y
end
f = [1, 2, 3, 4]
g = [5, 6, 7, 8]
l_show(L"f =", f, L"\; g =", g, L"\qquad f \circledast g =", circular_conv(f,g))

L"$f =$$\begin{equation}
\left(
\begin{array}{r}
1 \\
2 \\
3 \\
4 \\
\end{array}
\right)
\end{equation}
$$\; g =$$\begin{equation}
\left(
\begin{array}{r}
5 \\
6 \\
7 \\
8 \\
\end{array}
\right)
\end{equation}
$$\qquad f \circledast g =$$\begin{equation}
\left(
\begin{array}{r}
66 \\
68 \\
66 \\
60 \\
\end{array}
\right)
\end{equation}
$"

The circular convolution wraps entries

In [5]:
function linear_to_circular_conv( linear_result::Vector{T}, N::Int ) where T
    circular_result = zeros(T, N)
    for i in 1:length(linear_result)
        circular_result[mod1(i, N)] += linear_result[i]  # wrap the linear convolution result
    end
    circular_result
end
l_show(  L"\qquad f \circledast g =", linear_to_circular_conv(conv(f,g), length(f) ))

L"$\qquad f \circledast g =$$\begin{equation}
\left(
\begin{array}{r}
66 \\
68 \\
66 \\
60 \\
\end{array}
\right)
\end{equation}
$"

#### Example: Polynomials of a Circular Shift Matrix

The basic circular shift matrix $P$ of size $N \times N$ applied to a vector cyclically shifts the entries of the vector.

In [14]:
function circular_shift_matrix(N::Int)
    C = zeros(Int, N, N)
    for i in 1:N-1   C[i, i+1] = 1 end
    C[N, 1] = 1
    C
end
function circular_shift(vector::Vector)
    [vector[2:end]; vector[1]] # Shift the vector up by one position circularly
end

v = z[1:5] #[1,2,3,4,5]
P = circular_shift_matrix(length(v))
display(l_show( "circular shift matrix P = ", P, L"\quad v = ", v, L"\qquad Pv =", P*v, L"\qquad P^2v =", P^2*v))
@show P*v == circular_shift(v);

"\$\\text{circular shift matrix P = }\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n0 & 1 & 0 & 0 & 0 \\\\\n0 & 0 & 1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n0 & 0 & 0 & 0 & 1 \\\\\n1 & 0 & 0 & 0 & 0 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\quad v = \$\$\\begin{equation}\n\\left(\n\\begin{array" ⋯ 89 bytes ⋯ "quad Pv =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\nz_{1} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\qquad P^2v =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\nz_{1} \\\\\nz_{2} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$"

P * v == circular_shift(v) = true


____
Repeated application of $P$ shifts the vector further.

In [16]:
l_show(L"z =", v, L",\quad P z =", P*v,  L",\quad P^2 z =", P^2*v,
       L",\quad P^3 z =", P^3*v,  L",\quad P^4 z =", P^4*v, L",\quad P^5 z =", P^5*v)

"\$z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{1} \\\\\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$,\\quad P z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\nz_{1} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$,\\qua" ⋯ 280 bytes ⋯ "P^4 z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{5} \\\\\nz_{1} \\\\\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$,\\quad P^5 z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{1} \\\\\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$"

Given a vector of length $N$, repeated application of the shift matrix $P$ returns the vector to its original order in $N$ steps.

In [17]:
l_show(L"\quad P =", P,  L"\quad P^2 =", P^2,  L"\quad P^3 =", P^3,  L"\quad P^4 =", P^4, L"\quad P^5 =", P^5)

"\$\\quad P =\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n0 & 1 & 0 & 0 & 0 \\\\\n0 & 0 & 1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n0 & 0 & 0 & 0 & 1 \\\\\n1 & 0 & 0 & 0 & 0 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\quad P^2 =\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n0 & 0 & 1 & 0 " ⋯ 457 bytes ⋯ "1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\quad P^5 =\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n1 & 0 & 0 & 0 & 0 \\\\\n0 & 1 & 0 & 0 & 0 \\\\\n0 & 0 & 1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n0 & 0 & 0 & 0 & 1 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$"

Since  $P^N = I$ for a basic shift matrix of size $N\times N$, the multiplication of two polynomials in $P$<br>
can be expressed as a circular convolution of their respective coefficient vectors of size $N$. For example,

$\quad\left. \begin{align}
f(P) &= 3 I +2 P + P^3 \\
g(P) &= 2 P^3 \\
\end{align} \right\}\;\; \Rightarrow\;\;
\begin{pmatrix} 3\\ 2\\ 0\\ 1\\ 0\end{pmatrix} \circledast \begin{pmatrix}0\\ 0\\ 0\\ 2\\ 0 \end{pmatrix}
= \begin{pmatrix}0\\ 2\\ 0\\ 6\\ 4\end{pmatrix}\;\; \Rightarrow\;\;  f(P)\ g(P) = 2P + 6P^3+4 P^4$

In [9]:
@show circular_conv( [3, 2, 0, 1, 0], [0, 0, 0, 2, 0 ]) == [0,2,0,6,4];

circular_conv([3, 2, 0, 1, 0], [0, 0, 0, 2, 0]) == [0, 2, 0, 6, 4] = true


## 2. Circulant Matrices

In [10]:
function circular_shift_matrix_polynomial(c::Vector{T}) where T
    n = length(c)

    # Create the basic shift matrix S
    S = [i == mod(j+1, n) + 1 ? one(T) : zero(T) for i in 1:n, j in 1:n]

    # Create the polynomial
    p = Polynomial(c)

    # Evaluate the matrix polynomial
    C = p(S)

    return C
end

circular_shift_matrix_polynomial (generic function with 1 method)

# 3. Circular Shift Matrices and Circulant Matrices

[<strong>https://web.mit.edu/18.06/www/Spring17/Circulant-Matrices.pdf</strong>](https://web.mit.edu/18.06/www/Spring17/Circulant-Matrices.pdf)

In this lecture, I want to introduce you to a new type of matrix: circulant matrices. Like Hermitian
matrices, they have orthonormal eigenvectors, but unlike Hermitian matrices we know exactly what their
eigenvectors are! Moreover, their eigenvectors are closely related to the famous Fourier transform and Fourier
series. Even more importantly, it turns out that circulant matrices and the eigenvectors lend themselves to
incredibly efficient algorithms called FFTs, that play a central role in much of computational science and
engineering

IMPORTANT: $n$ independant values, not $n^2$.

## X.Y Remarks about Linear and Cyclic Convolution

**Key Contrasts:**
* **Matrix structure:** Linear convolution uses a Toeplitz matrix, while circular convolution uses a circulant matrix.
* **Length of result:** Linear convolution produces a longer output, while circular convolution maintains the input length.
* **Periodicity:** Circular convolution assumes the input signals are periodic, while linear convolution does not.
* **Eigenstructure:** Circulant matrices (used in circular convolution) have eigenvectors that are the Fourier basis vectors,<br>
simplifying computations in the frequency domain.

**Important Remark:** In practice, **linear convolution can be computed using circular convolution**<br>
$\qquad$ by zero-padding the input vectors to length $M+N-1$ before performing the circular convolution.<br>
$\qquad$ This relationship is crucial for efficient implementation of linear convolution using Fast Fourier Transform (FFT) algorithms.

# Example Stiffness Matrix

<div style="float:left;width:40%;">
Consider the following system consisting of<br>
$\qquad n$ masses, each of mass $m$<br>
$\qquad n$ springs, each with spring constant $k$<br>
where the masses are arranged in a circle, with each mass connected to its two neighbors
</div>
<div style="float:left;padding-left:1cm;">
<img src="Figs/CirculantMatrix_MassSpringSystem.svg" width=200>
</div>

In [11]:
function circular_spring_system(n::Int; m=1, k=1)
    # Obtain the stiffness matrix
    K = Matrix(Tridiagonal(fill(-k, n-1), fill(2k, n), fill(-k, n-1)))
    K[1, n] = K[n, 1] = -k  # Connect the ends

    K
end
K = circular_spring_system(7)
l_show("Stiffness Matrix ", L"\quad K =", K)

L"$\text{Stiffness Matrix }$$\quad K =$$\begin{equation}
\left(
\begin{array}{rrrrrrr}
2 & -1 & 0 & 0 & 0 & 0 & -1 \\
-1 & 2 & -1 & 0 & 0 & 0 & 0 \\
0 & -1 & 2 & -1 & 0 & 0 & 0 \\
0 & 0 & -1 & 2 & -1 & 0 & 0 \\
0 & 0 & 0 & -1 & 2 & -1 & 0 \\
0 & 0 & 0 & 0 & -1 & 2 & -1 \\
-1 & 0 & 0 & 0 & 0 & -1 & 2 \\
\end{array}
\right)
\end{equation}
$"

This matrix has a very special pattern: every row is the same as the previous row, just shifted to the
right by 1 (wrapping around “cyclically” at the edges). That is, each row is a circular shift of the first row.
This is called a circulant matrix.

$\qquad C_n = \begin{pmatrix} c_0     &     c_1 &     c_2 &     \dots & c_{n-1} \\
                       c_{n-1} &     c_0 &     c_1 &     \dots & c_{n-2} \\
                       c_{n-2} & c_{n-1} &     c_0 &     \dots & c_{n-3} \\
                       \dots   &  \dots  & \dots   &     \dots &  \dots  \\
                       c_1     &     c_2 &     c_3 &     \dots & c_0
\end{pmatrix}$
